**Defining Agent Types**

Daisy has three values aside from id and pos: age, breed (`:black` or `:white`), and albedo set by user. 
Land represents the surface, with the additional value of Temperature. Since the entire surface has the same albedo field, we don't need to set it as a Land value. 

*Note: Land does not necessarily have to be an agent, and you can represent surface temperature as a matrix (model parameter). This gives a performance advantage,*
*but incorporating surface agents allows for easier visualization*

In [2]:
using Agents, Plots 
using Statistics: mean

mutable struct Daisy <: AbstractAgent
    id::Int
    pos::Dims{2}
    breed::Symbol
    age::Int
    albedo::Float64 #0-1 Fraction
end

mutable struct Land <: AbstractAgent
    id::Int
    pos::Dims{2}
    temperature::Float64
end

const DaisyWorld = ABM{<:GridSpace, Union{Daisy, Land}};

**World Heating**

Surface temperature of the world is heated by the sun, but the daisies growing on the surface will either reflect or absorb the sunlight, altering the surface temperature

In [3]:
function update_surface_temperature(pos::Dims{2}, model::DaisyWorld)
    ids = ids_in_position(pos, model)
    #All grid positions have at least one agent (land) = Set luminosity via surface albedo
    #More than one agent means daisy exists = Set luminosity via daisy albedo
    absorbed_luminosity = 
    (length(ids) == 1) ? (1-model.surface_albedo)*model.solar_liminosity : (1-model[ids[2]].albedo) * model.solar_liminosity
    #We expect heating to be 80 °C for an absorbed luminosity of 1,
    #approximately 30 for 0.5 and approximately -273 for 0.01.
    local_heating = (absorbed_luminosity > 0) ? 72 * log(absorbed_luminosity) + 80 : 80
    #Surface temperature is the average of the current temperature and local heating
    T0 = model[ids[1]].temperature 
    model[ids[1]].temperature = (T0 + local_heating) / 2
end


update_surface_temperature (generic function with 1 method)

Additionally, temperature diffuses over time.

In [4]:
function diffuse_temperature!(pos::Dims{2}, model::DaisyWorld)
    ratio = get(model.properties, :ratio, 0.5) #diffusion ratio
    ids = nearby_ids(pos, model)
    meantemp = sum(model[i].temperature for i in ids if model[i] isa Land) / 8
    land = model[ids_in_position(pos, model)[1]] #land at current position 
    #each neighbor land patch is giving up 1/8 of the diffused amount to each of its neighbors
    land.temperature = (1 - ratio) * land.temperature + ratio * meantemp
end


diffuse_temperature! (generic function with 1 method)

**Daisy Dynamics**

Function below determines the optimal temperature for growth. If temperature is too hot or too cold, Daisies will not wish to propagate. As long as temperature is optimal, Daisies will compete for land and create new Daisies of their `breed` close to them. 

In [5]:
function propagate!(pos::Dims{2}, model::DaisyWorld)
    ids = ids_in_position(pos, model)
    if length(ids) > 1
        daisy = model[ids[2]]
        temperature = model[ids[1]].temperature
        # Set optimum growth rate to 22.5 °C, with bounds of [5, 40]
        seed_threshold = (0.1457 * temperature - 0.0032 * temperature^2) - 0.6443
        if rand() < seed_threshold
            #Collect all adjacent positions that have no daisies
            empty_neighbors = Tuple{Int, Int}[]
            neighbors = nearby_positions(pos, model)
            for n in neighbors
                if length(ids_in_position(n, model)) == 1
                    push!(empty_neighbors, n)
                end
            end
            if !isempty(empty_neighbors)
                #Seed a new daisy in one of those position
                seeding_place = rand(empty_neighbors)
                a = Daisy(nextid(model), seeding_place, daisy.breed, 0, daisy.albedo)
                add_agent_pos!(a, model) #add agent to its own position
            end
        end
    end
end



propagate! (generic function with 1 method)

If Daisies cross the age threshold, they die out. Death is controlled by the `agent_step` function

In [6]:
function agent_step!(agent::Daisy, model::DaisyWorld)
    agent.age += 1
    agent.age ≥ model.max_age && kill_agent!(agent, model)
end


agent_step! (generic function with 1 method)

Must define `agent_step` function for Land agent as well. Land dynamics are resolved at model level, so this step function won't do anything.

In [11]:
agent_step!(agent::Land, model::DaisyWorld) = nothing

agent_step! (generic function with 2 methods)

`model_step` function advances the Daisyworld model dynamics. If We have defined a number of helper functions above, so this function is only matter of calling these helper functions. 

In [9]:
function model_step!(model)
    for p in positions(model)
        update_surface_temperature!(p, model)
        diffuse_temperature!(p, model)
        propagate!(p, model)
    end
    model.tick += 1
    solar_activity!(model)
end


model_step! (generic function with 1 method)

`solar_activity` function changes the amount of incoming solar radiation over time (if the "scenario" parameter is `:ramp`). Model parameter tick keeps track of time

In [12]:
function solar_activity!(model::DaisyWorld)
    if model.scenario == :ramp
        if model.tick > 200 && model.tick ≤ 400
            model.solar_luminosity += model.solar_change
        end
        if model.tick > 500 && model.tick <= 750
            model.solar_luminosity -= model.solar_change / 2
        end
    elseif model.scenario == :change
        model.solar_luminosity += model.solar_change
    end
end

solar_activity! (generic function with 1 method)

**Initialising Daisyworld**

Here, we construct a function to initialize Daisyworld. First, we use `fill_space!` to fill grid spaces with `Land` agents. Next, we seed planet with daisies of each type and define their albedos. 

In [26]:
import StatsBase

function daisyworld(;
    griddims = (30,30),
    max_age = 25,
    init_white = 0.2, # %cover of the world surface of white breed
    init_black = 0.2, # %cover of world surface of black breed
    albedo_white = 0.75,
    albedo_black = 0.25,
    surface_albedo = 0.4,
    solar_change = 0.005,
    solar_luminosity = 1.0, #initial solar_luminosity
    scenario = :default,
)

    space = GridSpace(griddims)
    properties = Dict(:max_age => max_age, :surface_albedo => surface_albedo, :solar_luminosity => solar_luminosity, :solar_change => solar_change, :scenario => scenario)
    properties[:tick] = 0
    #create a scheduler that only schedules Daisies
    daisysched(model) = [a.id for a in allagents(model) if a isa Daisy]
    model = ABM(
        Union{Daisy, Land},
        space;
        scheduler = daisysched,
        properties = properties,
        warn = false,
    )

    # fill model with `Land` : eveery grid position has one land instance
    fill_space!(Land, model, 0.0) #zero starting temperature

    #Populate with daisies: each position has only one daisy (black or white)
    grid = collect(positions(model))
    num_positions = prod(griddims) #product of dimmensions. Finds area of grid
    white_positions = 
        StatsBase.sample(grid, Int(init_white * num_positions); replace = false)
    for wp in white_positions
        wd = Daisy(nextid(model), wp, :white, rand(0:max_age),albedo_white)
        add_agent_pos!(wd, model)
    end
    allowed = setdiff(grid, white_positions)
    black_positions = 
        StatsBase.sample(allowed, Int(init_black * num_positions); replace = false)
    for bp in black_positions
        wd = Daisy(nextid(model), bp, :black, rand(0:max_age), albedo_black)
        add_agent_pos!(wd, model)
    end

    return model
end


daisyworld (generic function with 1 method)

**Visualizing & Animating**


In [27]:
model = daisyworld()

AgentBasedModel with 1260 agents of type Union{Daisy, Land}
 space: GridSpace with size (30, 30), metric=chebyshev, periodic=true
 scheduler: daisysched
 properties: solar_luminosity, max_age, surface_albedo, solar_change, tick, scenario

In [30]:
daisycolor(a::Daisy) = a.breed
const landcolor = cgrad(:thermal)
daisycolor(a::Land) = landcolor[(a.temperature+50)/50]

daisycolor (generic function with 2 methods)

In [32]:
daisyshape(a::Daisy) = :circle
daisysize(a::Daisy) = 7 
daisyshape(a::Land) = :square
daisysize(a::Land) = 8.8


daisysize (generic function with 2 methods)

In [42]:
using InteractiveDynamics
using CairoMakie

CairoMakie.activate!()

plotsched = Schedulers.ByType((Land, Daisy), false)

plotkwargs = (
    ac = daisycolor,
    am = daisyshape, 
    as = daisysize, 
    scheduler = plotsched,
    aspect_ratio = 1, 
    size = (600, 600),
    showaxis = false,
)

fig, ax, abmobs = abmplot(model; plotkwargs...)
fig

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

...

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa

┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packages/Makie/6TJFL/src/conversions.jl:1231
┌ Warning: Unsupported marker: square, using ● instead
└ @ Makie /Users/pbb62/.julia/packa